In [1]:
import numpy as np
import pandas as pd
import json

pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv('trainv2_clean.csv')

/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cont = ['visitNumber', 'visitStartTime', 'sessionQualityDim', 'pageviews', 'transactionRevenue', 'timeOnSite', 
        'transactions', 'transactionRevenue', 'totalTransactionRevenue']
cat = ['newVisits', 'bounces', 'channelGrouping', 'browser', 'operatingSystem', 'isMobile', 
       'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'city', 'networkDomain', 'customDim', 
       'campaign', 'source', 'medium', 'keyword', 'isTrueDirect']

In [4]:
train_df = pd.DataFrame(np.zeros((len(df), len(cont)+len(cat)+1)), columns=(cont+cat+['fullVisitorId']))

In [5]:
df['transactionRevenue'] = np.log(df['transactionRevenue'].values)
df = df.replace(np.log(0), 0)
tr = {'mean': 0, 'std': 0}
for col in cont:
    print(col)
    m = df[col].values.mean()
    sd = df[col].values.std()
    if(col == 'transactionRevenue'):
        tr['mean'] = m
        tr['std'] = sd
    train_df[col] = (df[col].values-m) / sd
for col in cat:
    train_df[col] = df[col].values
train_df['fullVisitorId'] = df['fullVisitorId'].values

/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


visitNumber
visitStartTime
sessionQualityDim
pageviews
transactionRevenue
timeOnSite
transactions
transactionRevenue
totalTransactionRevenue


In [6]:
train_df.head()

,visitNumber,visitStartTime,sessionQualityDim,pageviews,transactionRevenue,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue,newVisits,bounces,channelGrouping,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain,customDim,campaign,source,medium,keyword,isTrueDirect,fullVisitorId
0,-0.142737,0.605943,-0.109881,-0.416455,-0.104437,-0.349443,-0.097997,-0.104437,-0.022493,1,1,Organic Search,Firefox,Windows,False,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,(not set),EMEA,(not set),google,organic,water bottle,False,3162355547410993243
1,0.391791,0.604581,0.010105,-0.261966,-0.104437,-0.270448,-0.097997,-0.104437,-0.022493,0,0,Referral,Chrome,Chrome OS,False,desktop,Americas,Northern America,United States,California,Cupertino,(not set),North America,(not set),sites.google.com,referral,[NOT GIVEN],False,8934116514970143966
2,-0.142737,0.606138,-0.109881,-0.261966,-0.104437,-0.242236,-0.097997,-0.104437,-0.022493,1,0,Direct,Chrome,Android,True,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,windjammercable.net,North America,(not set),(direct),(none),[NOT GIVEN],True,7992466427990357681
3,-0.142737,0.604183,-0.109881,-0.261966,-0.104437,-0.346622,-0.097997,-0.104437,-0.022493,1,0,Organic Search,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,not available in demo dataset,not available in demo dataset,unknown.unknown,EMEA,(not set),google,organic,(not provided),False,9075655783635761930
4,-0.142737,0.605457,-0.109881,-0.261966,-0.104437,-0.202739,-0.097997,-0.104437,-0.022493,1,0,Organic Search,Chrome,Windows,False,desktop,Americas,Central America,Mexico,not available in demo dataset,not available in demo dataset,prod-infinitum.com.mx,Central America,(not set),google,organic,(not provided),False,6960673291025684308


In [7]:
train_df.to_csv('trainv2_df.csv', index=False)

In [9]:
no_t = train_df.transactions.unique()[0]

In [13]:
bought = train_df[train_df['transactions']!=no_t]
num_buy = bought.shape[0]
did_not = train_df[train_df['transactions']==no_t].sample(num_buy*9)
train_10 = pd.concat((bought,did_not)).sample(frac=1)

In [25]:
train_10['bought'] = np.equal(train_10.transactions.values, no_t).astype(np.uint8)
train_10['bought'] = train_10['bought'].apply(lambda x: 1 if x==0 else 0)

In [26]:
print(train_10.shape)
train_10.head()

(185590, 30)


,visitNumber,visitStartTime,sessionQualityDim,pageviews,transactionRevenue,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue,newVisits,bounces,channelGrouping,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain,customDim,campaign,source,medium,keyword,isTrueDirect,fullVisitorId,bought
771192,-0.142737,1.460345,-0.109881,0.047014,-0.104437,-0.278912,-0.097997,-0.104437,-0.022493,1,0,Organic Search,Chrome,Windows,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,rr.com,North America,(not set),google,organic,(not provided),False,6056611153860034794,0
796041,-0.035832,1.060616,-0.109881,-0.416455,-0.104437,-0.349443,-0.097997,-0.104437,-0.022493,0,1,Organic Search,Chrome,Macintosh,False,desktop,Asia,Southeast Asia,Singapore,(not set),(not set),myrepublic.com.sg,APAC,(not set),google,organic,(not provided),True,5062502445577894046,0
808271,-0.142737,0.928317,-0.109881,-0.416455,-0.104437,-0.349443,-0.097997,-0.104437,-0.022493,1,1,Social,Chrome,Windows,False,desktop,Europe,Eastern Europe,Russia,not available in demo dataset,not available in demo dataset,unknown.unknown,EMEA,(not set),youtube.com,referral,[NOT GIVEN],False,7144515002115728466,0
976050,-0.142737,0.059578,-0.109881,0.047014,-0.104437,-0.194275,-0.097997,-0.104437,-0.022493,1,0,Direct,Chrome,Linux,False,desktop,Asia,Southern Asia,India,Delhi,(not set),unknown.unknown,APAC,(not set),(direct),(none),[NOT GIVEN],True,1776496258518913179,0
127792,-0.142737,-0.037400,-0.229867,-0.261966,-0.104437,-0.338158,-0.097997,-0.104437,-0.022493,1,0,Organic Search,Safari,iOS,True,mobile,Americas,Northern America,United States,California,San Francisco,(not set),North America,(not set),(direct),(none),[NOT GIVEN],False,9045651405070470290,0


In [28]:
train_10.drop(['transactionRevenue', 'transactions', 'totalTransactionRevenue'], axis=1, inplace=True)
train_10.head()

,visitNumber,visitStartTime,sessionQualityDim,pageviews,timeOnSite,newVisits,bounces,channelGrouping,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain,customDim,campaign,source,medium,keyword,isTrueDirect,fullVisitorId,bought
771192,-0.142737,1.460345,-0.109881,0.047014,-0.278912,1,0,Organic Search,Chrome,Windows,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,rr.com,North America,(not set),google,organic,(not provided),False,6056611153860034794,0
796041,-0.035832,1.060616,-0.109881,-0.416455,-0.349443,0,1,Organic Search,Chrome,Macintosh,False,desktop,Asia,Southeast Asia,Singapore,(not set),(not set),myrepublic.com.sg,APAC,(not set),google,organic,(not provided),True,5062502445577894046,0
808271,-0.142737,0.928317,-0.109881,-0.416455,-0.349443,1,1,Social,Chrome,Windows,False,desktop,Europe,Eastern Europe,Russia,not available in demo dataset,not available in demo dataset,unknown.unknown,EMEA,(not set),youtube.com,referral,[NOT GIVEN],False,7144515002115728466,0
976050,-0.142737,0.059578,-0.109881,0.047014,-0.194275,1,0,Direct,Chrome,Linux,False,desktop,Asia,Southern Asia,India,Delhi,(not set),unknown.unknown,APAC,(not set),(direct),(none),[NOT GIVEN],True,1776496258518913179,0
127792,-0.142737,-0.037400,-0.229867,-0.261966,-0.338158,1,0,Organic Search,Safari,iOS,True,mobile,Americas,Northern America,United States,California,San Francisco,(not set),North America,(not set),(direct),(none),[NOT GIVEN],False,9045651405070470290,0


In [32]:
train_10.drop('networkDomain', axis=1, inplace=True)

In [34]:
train_10.to_csv("trainv2_10.csv", index=False)

In [18]:
train_df.transactionRevenue.value_counts()

-0.113349     892138
 8.195309        256
 8.250850        189
 8.535480        187
 8.679092        170
 8.083870        135
 8.790479        122
 8.276462        116
 8.165035         98
 8.139419         93
 8.256079         92
 7.977893         84
 8.824918         81
 8.387877         77
 8.968513         65
 8.429986         64
 8.429802         62
 8.541241         54
 8.622519         51
 8.881412         46
 7.125049         44
 8.458349         40
 8.324052         40
 8.569784         39
 8.485515         39
 8.212630         38
 8.061335         37
 8.444280         37
 8.485351         37
 8.511123         37
               ...  
 10.073901         1
 9.105999          1
 10.002850         1
 8.748082          1
 9.216302          1
 9.546612          1
 9.011470          1
 9.574815          1
 9.095419          1
 9.125052          1
 10.095632         1
 9.547513          1
 8.803674          1
 8.637275          1
 8.942385          1
 10.187575         1
 9.768927    